In [50]:
from __future__ import division
from numba import cuda
from numba import jit
import numpy
import math
import os
pygad_loc = '/grid/home/nbourgeois/codonOpt/'
os.chdir(pygad_loc)
from metrics import *
from general_functions import *
from codon_tools import *
from metric_generator import *

ERROR! Session/line number was not unique in database. History logging moved to new session 3991


In [17]:
pah ='MSTAVLENPGLGRKLSDFGQETSYIEDNCNQNGAISLIFSLKEEVGALAKVLRLFEENDVNLTHIESRPSRLKKDEYEFFTHLDKRSLPALTNIIKILRHDIGATVHELSRDKKKDTVPWFPRTIQELDRFANQILSYGAELDADHPGFKDPVYRARRKQFADIAYNYRHGQPIPRVEYMEEEKKTWGTVFKTLKSLYKTHACYEYNHIFPLLEKYCGFHEDNIPQLEDVSQFLQTCTGFRLRPVAGLLSSRDFLGGLAFRVFHCTQYIRHGSKPMYTPEPDICHELLGHVPLFSDRSFAQFSQEIGLASLGAPDEYIEKLATIYWFTVEFGLCKQGDSIKAYGAGLLSSFGELQYCLSEKPKLLPLELEKTAIQNYTVTEFQPLYYVAESFNDAKEKVRNFAATIPRPFSVRYDPYTQRIEVLDNTQQLKILADSINSEIGILCSALQKIK'

In [62]:
seqs = gen_random_seqs(pah,100000)

In [56]:
mg = MetricGenerator('Liver')

In [63]:
%%time 
bais = [get_bai(seq,mg.bai_weight_dict) for seq in seqs]

CPU times: user 9.79 s, sys: 339 ms, total: 10.1 s
Wall time: 10.1 s


In [60]:
bais

[0.27787846847427167,
 0.2762487444584984,
 0.2799292834447851,
 0.27475928580813175,
 0.26718010503464346,
 0.26747164053851963,
 0.27350595376826653,
 0.29482917394102176,
 0.28391831463936634,
 0.266668135044732,
 0.2819500434237637,
 0.263577845679655,
 0.27756576429567636,
 0.2694932058144455,
 0.2929719715111795,
 0.28405523935229676,
 0.29322742129509466,
 0.30066554626955033,
 0.2824404153747169,
 0.29550358812184174,
 0.2704868694815502,
 0.29142243113517213,
 0.26451199126644953,
 0.2936386953409977,
 0.29762248041261474,
 0.2983236190435783,
 0.27043165007311365,
 0.2936315686803858,
 0.2875729238374581,
 0.28327013861308425,
 0.2701931578986381,
 0.2919614513615013,
 0.30126848417247604,
 0.2710040401775325,
 0.2741750252421309,
 0.2552181852241521,
 0.2767477300622375,
 0.28282021345851005,
 0.28269462823011776,
 0.2707407007588741,
 0.26408463218689576,
 0.2765416971199036,
 0.28859530273022005,
 0.29638847343133656,
 0.26821113960027204,
 0.2822988079789052,
 0.294874689

In [11]:
%%time 
# Create the data array - usually initialized some other way
data = numpy.ones(256)

# Set the number of threads in a block
threadsperblock = 1238 

# Calculate the number of thread blocks in the grid
blockspergrid = (data.size + (threadsperblock - 1)) // threadsperblock

# Now start the kernel
my_kernel[blockspergrid, threadsperblock](data)

# Print the result
print(data)

[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
CPU times: user 8.37 ms, sys: 1.18 ms, total: 9.55 ms
Wall time: 7.34 ms


In [ ]:
%%time

# CUDA kernel
@cuda.jit
def my_kernel(io_array):
    pos = cuda.grid(1)
    if pos < io_array.size:
        io_array[pos] *= 2 # do the computation

# Host code   
data = numpy.ones(102800000)
threadsperblock = 512
blockspergrid = math.ceil(data.shape[0] / threadsperblock)
for i in range(100):
    data = numpy.ones(102800000)

    my_kernel[blockspergrid, threadsperblock](data)
print(data)

/grid/home/nbourgeois/envs/bfx-codon/lib/python3.7/site-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/grid/home/nbourgeois/envs/bfx-codon/lib/python3.7/site-packages/IPython/core/magics/execution.py", line 1321, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 13, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/grid/home/nbourgeois/envs/bfx-codon/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/grid/home/nbourgeois/envs/bfx-codon/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/grid/home/nbourgeois/envs/bfx-codon/lib/python3.7/site-packages/IPytho

In [46]:
%%time

data = numpy.ones(1028000000)

data = data*2

CPU times: user 3.14 s, sys: 10.4 s, total: 13.6 s
Wall time: 13.6 s


In [5]:
%%time
# CUDA kernel
@jit
def my_kernel(io_array):
    pos = cuda.grid(1)
    if pos < io_array.size:
        io_array[pos] *= 2 # do the computation

# Host code   
data = numpy.ones(256)
threadsperblock = 256
blockspergrid = math.ceil(data.shape[0] / threadsperblock)
my_kernel[blockspergrid, threadsperblock](data)
print(data)

TypeError: 'CPUDispatcher' object is not subscriptable